In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

Обробка(об'єднання) вихідних даних

In [37]:
from pathlib import Path
data_features = ['idle', 'running', 'stairs', 'walking']
data = []

for i, feature in enumerate(data_features):
    activity_path = Path('data', feature)
    csv_files = list(activity_path.glob('*.csv'))
    df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
    df['activity'] = feature
    data.append(df)
    
activity_data = pd.concat(data, ignore_index=True)
activity_data.head(10)       

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle
5,-0.086191,-0.148441,9.787497,idle
6,-0.062249,-0.191536,9.773131,idle
7,-0.057461,-0.114922,9.763555,idle
8,-0.124498,-0.162806,9.782708,idle
9,-0.138864,-0.124498,9.768343,idle


Нормалізація даних, створення копії оригінальних даних 

In [38]:
features = ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']
scaler = StandardScaler()
data_to_norm = activity_data[features]
normalized_data = scaler.fit_transform(data_to_norm)

new_data = activity_data.copy()
new_data[features] = normalized_data
new_data.head(10)

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-0.109791,0.241917,0.941538,idle
1,-0.143404,0.209288,0.923560,idle
2,-0.337109,-0.150782,1.028762,idle
3,0.377889,-0.116234,0.919565,idle
4,-0.019205,-0.141185,1.035420,idle
5,-0.239117,-0.140034,1.109994,idle
6,-0.236268,-0.143489,1.107996,idle
7,-0.235699,-0.137347,1.106665,idle
8,-0.243675,-0.141185,1.109328,idle
9,-0.245384,-0.138114,1.107331,idle


Розділення даних на тестові та тренувальні набори

In [39]:
X_train, X_test, y_train, y_test = train_test_split(new_data[features], new_data['activity'], test_size=0.3, random_state=42)

Навчання на тренувальних даних за алгоритмом SVM

In [40]:
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_model.score(X_test, y_test)

0.8928264383231885

Навчання на тренувальних даних за алгоритмом Random Forest

In [41]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)

0.9995873310636542

Передбачення

In [42]:
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

Звіт класифікації

In [46]:
svm_report = classification_report(y_test, svm_predictions)
print(svm_report)


              precision    recall  f1-score   support

        idle       0.96      0.98      0.97      9306
     running       0.93      0.91      0.92     30609
      stairs       1.00      0.01      0.01      1537
     walking       0.80      0.90      0.85     16706

    accuracy                           0.89     58158
   macro avg       0.92      0.70      0.69     58158
weighted avg       0.90      0.89      0.88     58158



In [47]:
rf_report = classification_report(y_test, rf_predictions)
print(rf_report)

              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      9306
     running       1.00      1.00      1.00     30609
      stairs       1.00      0.99      1.00      1537
     walking       1.00      1.00      1.00     16706

    accuracy                           1.00     58158
   macro avg       1.00      1.00      1.00     58158
weighted avg       1.00      1.00      1.00     58158



## Висновки: 
Модель випадкового лісу демонструє високу точність, що наближається до 100%. У моделі опорних векторів точність менша, приблизно 89%. В моделі випадкового лісу значення precision та f1-score становлять 1, що свідчить про високу точність передбачення класів. У моделі опорних векторів спостерігаються гірші результати, precision показує недостатню точність, що може вказувати на наявність помилок у передбаченнях. 